# Retrieval Augmented Generation (RAG) Server

## 1. Setup

**Instructions:**

a) Download model

```bash
huggingface-cli download hkunlp/instructor-large \
    --revision 54e5ffb8d484de506e59443b07dc819fb15c7233 \
    --local-dir ~/.gai/models/instructor-large \
    --local-dir-use-symlinks False
```


---

## 2. Load test configuration

In [1]:
from gai.lib.server.singleton_host import SingletonHost
from gai.lib.common.utils import free_mem
from rich.console import Console
console=Console()

config = {
    "type": "rag",
    "generator_name": "instructor-sentencepiece",
    "chromadb": {
        "path": "rag/chromadb",
        "n_results": 3
    },
    "sqlite": {
        "path": "rag/gai-rag.db"
    },
    "model_path": "models/instructor-large",
    "device": "cuda",
    "chunks": {
        "size": 1000,
        "overlap": 100,
        "path": "chunks"
    },
    "module_name": "gai.rag.server.gai_rag",
    "class_name": "RAG",
    "init_args": [],
    "init_kwargs": {}
}


## 3. Load Model Test

In [2]:
# before loading
free_mem()
try:
    with SingletonHost.GetInstanceFromConfig(config) as host:

        # after loading
        free_mem()
except Exception as e:
    raise e
finally:
    # after disposal
    free_mem()
    

Free memory: 5.66 GB

INFO     RAG: device=cuda
INFO     RAG: sqlite=sqlite:///:memory:


load INSTRUCTOR_Transformer


/home/kakkoii1337/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


max_seq_length  512


                         Performance load                          
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 1.15 seconds │ 1.15 seconds │
│ CPU Memory        │          - MB │      5.75 MB │      5.75 MB │
│ CUDA Memory       │       5.69 GB │      0.00 GB │      5.69 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

Free memory: 5.69 GB

Free memory: 5.69 GB

## Indexing

In [2]:
from gai.rag.server.gai_rag import RAG
from gai.rag.server.dtos.create_doc_header_request import CreateDocHeaderRequestPydantic
try:
    with SingletonHost.GetInstanceFromConfig(config) as host:
        rag = host.generator

        req = CreateDocHeaderRequestPydantic(
            CollectionName='demo',
            FileType='txt',
            Source='https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech',
            Title='2023 National Day Rally Speech',
            FilePath="./pm_long_speech_2023.txt"
        )


        # Index
        chunkids = await rag.index_async(
            req=req
            # collection_name='demo',
            # file_type='txt',
            # source="https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech",
            # title="2023 National Day Rally Speech"
            )

except Exception as e:
    raise e
finally:
    # after disposal
    free_mem()


INFO     RAG: device=cuda
INFO     RAG: sqlite=sqlite:///:memory:


load INSTRUCTOR_Transformer
max_seq_length  512


/home/kakkoii1337/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


                         Performance load                          
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 1.28 seconds │ 1.28 seconds │
│ CPU Memory        │          - MB │      5.72 MB │      5.72 MB │
│ CUDA Memory       │       5.68 GB │      0.01 GB │      5.67 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

INFO     rag.index_document_header_async: request started. collection_name=demo file_path=./pm_long_speech_2023.txt title=2023 National Day Rally Speech source=https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech abstract=None authors=None publisher=None published_date=None comments=None keywords=None
DEBUG    rag.index_document_header_async: creating doc header with id=PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U.
DEBUG    rag.index_document_header_async: document_header created. id=PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U
INFO     rag.index_document_split_async: splitting chunks
INFO     rag.index_document_split_async: chunkgroup created. chunkgroup_id=a09d97b0-7246-4c9a-b564-af39428049da
100%|██████████| 66/66 [00:00<00:00, 533.84it/s]
INFO     rag.index_document_split_async: chunks created. count=66
INFO     RAG.index_document_index_async: Start indexing...
0it [00:00, ?it/s]DEBUG    RAG.index_document_index_async: Indexed 1/66 chunk afad2144-e170-4932-9613-b33ef1ed17c0

Free memory: 5.53 GB

In [3]:
try:
    with SingletonHost.GetInstanceFromConfig(config) as host:
        rag=host.generator
        # Index
        result = rag.retrieve(collection_name="demo",query_texts="Who are the young seniors?")
        console.print(result)

except Exception as e:
    raise e
finally:
    # after disposal
    free_mem()


INFO     RAG: device=cuda
INFO     RAG: sqlite=sqlite:///:memory:


load INSTRUCTOR_Transformer
max_seq_length  512


                         Performance load                          
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 0.75 seconds │ 0.75 seconds │
│ CPU Memory        │          - MB │      1.48 MB │      1.48 MB │
│ CUDA Memory       │       5.53 GB │      0.00 GB │      5.53 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

INFO     RAG.retrieve: Retrieving by query Who are the young seniors?...


[
    {
        'documents': 'Especially for those in their 50s and early 60s. Let us call them the “Young Seniors”. 
"Young”, because you are younger than the Pioneer Generation and the Merdeka Generation; “Seniors”, because you 
will soon retire, or maybe you have recently retired.',
        'metadatas': {
            'Abstract': '',
            'ChunkGroupId': 'a09d97b0-7246-4c9a-b564-af39428049da',
            'DocumentId': 'PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U',
            'Keywords': '',
            'PublishedDate': '',
            'Source': 'https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech',
            'Title': '2023 National Day Rally Speech'
        },
        'distances': 0.09020859003067017,
        'ids': 'dcd5d1d2-e234-4c0b-bbf2-ab040b685b1b'
    },
    {
        'documents': 'Young Seniors are in a unique position today. Compared to the Pioneer and Merdeka 
Generations, you have benefited more from Singapore’s growth, and generally done better in life. But compared to 
workers younger than you, in their 30s and 40s today, you have generally earned less over your lifetimes. You have 
also had less time to benefit from improvements to the CPF system, and so have built up less retirement savings. 
Young Seniors are also in a particularly sandwiched phase of your lives. You have to shoulder the responsibility of
caring for both the young and old in your families. Your kids may be young adults, but often are not yet fully 
independent and still live in the same household. Many of you Young Seniors also have elderly parents at home, who 
may be beset with the infirmities of old age. You have to shuttle them to medical appointments and hospital visits,
plus attend to their everyday needs. All this, while watching your own health, because you are not so youthful',
        'metadatas': {
            'Abstract': '',
            'ChunkGroupId': 'a09d97b0-7246-4c9a-b564-af39428049da',
            'DocumentId': 'PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U',
            'Keywords': '',
            'PublishedDate': '',
            'Source': 'https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech',
            'Title': '2023 National Day Rally Speech'
        },
        'distances': 0.11518943309783936,
        'ids': 'd16464e0-2915-4c1c-8efb-909ae724b38a'
    },
    {
        'documents': 'to their everyday needs. All this, while watching your own health, because you are not so 
youthful yourselves. Responsibilities on all these fronts multiply your burdens. So, as Young Seniors, you are 
understandably anxious about your retirement needs. Beyond the daily cost of living pressures, you know that 
retirement is creeping up on you. You wonder: Will I have enough to get by? Can I cope? But do not worry – the 
Government will help you. You will not be left behind.',
        'metadatas': {
            'Abstract': '',
            'ChunkGroupId': 'a09d97b0-7246-4c9a-b564-af39428049da',
            'DocumentId': 'PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U',
            'Keywords': '',
            'PublishedDate': '',
            'Source': 'https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech',
            'Title': '2023 National Day Rally Speech'
        },
        'distances': 0.11691617965698242,
        'ids': 'd1fb6e71-f041-4c71-b24b-031377281198'
    }
]

Free memory: 5.53 GB

---

## API (Press F5 to start API server)

Wait for it complete loading before running the next cell.

#### a) List Collections

In [1]:
%%bash
curl -s http://localhost:12036/gen/v1/rag/collections

{"collections":[]}

#### b) Delete collection

In [2]:
%%bash
curl -s -X DELETE http://localhost:12036/gen/v1/rag/collection/demo

{"detail":{"code":"collections_not_found","message":"Collection demo not found"}}

#### c) index

In [3]:
%%bash
curl -X POST 'http://localhost:12036/gen/v1/rag/index-file' \
    -H 'accept: application/json' \
    -H 'Content-Type: multipart/form-data' \
    -s \
    -F 'file=@./pm_long_speech_2023.txt' \
    -F 'req={"CollectionName":"demo","Source": "https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2023","FilePath":"./pm_long_speech_2023.txt"}'

{"DocumentId":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","ChunkgroupId":"8f677904-195a-45ff-b001-4f0cfaeceb7f","ChunkIds":["2b51573f-5cdc-49cb-84e9-f9e54b5e1d2c","2100e210-721b-48ef-b038-a9fec1277141","412965e9-3d2b-4613-9626-1a40a72ac353","19bd08b0-2b1f-462a-ae5b-72f1036519f4","8eac6941-3c63-451b-bbde-89a47af13d8e","f2efae9a-8207-4a4e-b150-89790601d858","870bba2c-858d-4545-b1d8-abacbbcbadcc","781ff509-a53d-47a3-a55e-a96a69f27fc5","a401e210-f453-4c85-82b2-127903c55323","d82f17b6-334e-48cb-a7cf-c8ae0c46cdee","580ffa78-a875-40a6-b1c8-ed722e899e71","2946f817-6b2b-4d74-b57d-1d043c8538fd","98a00490-6f77-4419-ba0b-29c0304b36f5","951e7e93-f7de-40c7-89b1-009cf88df5d1","678a8955-463a-4652-9e10-a89a46db0510","895ba232-7ec1-46f0-b351-9bc234caf421","4ae2f814-3a2c-419e-8de1-4da68f869be9","eddde16b-1c6b-41f6-8409-4d2800170ce9","80f15407-e225-47f7-a19d-8b093b2aef9e","559bd7fb-c551-46b2-bd61-ad5a4e578574","e66ec4dc-a481-452e-99c5-4e0b297452e4","30dca5db-3d30-45bd-afde-10ac711fd06b","8e69e591-1c2c-4

#### d) verify document exists

In [4]:
%%bash
curl -X POST 'http://localhost:12036/gen/v1/rag/collection/demo/document/exists' \
    -H 'accept: application/json' \
    -H 'Content-Type: multipart/form-data' \
    -s \
    -F 'file=@./pm_long_speech_2023.txt' 

{"Id":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","CollectionName":"demo","ByteSize":43352,"FileName":"pm_long_speech_2023.txt","FileType":".txt","File":null,"Source":"https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2023","Abstract":null,"Authors":null,"Title":null,"Publisher":null,"PublishedDate":null,"Comments":null,"Keywords":null,"CreatedAt":"2024-09-17T11:38:14.739628","UpdatedAt":"2024-09-17T11:38:14.739638","IsActive":true,"ChunkGroups":[{"Id":"8f677904-195a-45ff-b001-4f0cfaeceb7f","DocumentId":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","SplitAlgo":"recursive_split","ChunkCount":66,"ChunkSize":1000,"Overlap":100,"IsActive":true,"ChunksDir":"/tmp/chunks/d89830d0a8894e4ba3d4833938ab0a7b"}]}

#### e) retrieve

In [6]:
%%bash
curl -X POST 'http://localhost:12036/gen/v1/rag/retrieve' \
    -s \
    -H "Content-Type: application/json" \
    -d '{"collection_name":"demo","query_texts":"Who are the young seniors?","n_results":4}'


{"retrieved":[{"documents":"Especially for those in their 50s and early 60s. Let us call them the “Young Seniors”. \"Young”, because you are younger than the Pioneer Generation and the Merdeka Generation; “Seniors”, because you will soon retire, or maybe you have recently retired.","metadatas":{"Abstract":"","ChunkGroupId":"8f677904-195a-45ff-b001-4f0cfaeceb7f","DocumentId":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","Keywords":"","PublishedDate":"","Source":"https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2023","Title":""},"distances":0.09020859003067017,"ids":"e79a432a-c83e-4460-8979-799679e295b8"},{"documents":"Young Seniors are in a unique position today. Compared to the Pioneer and Merdeka Generations, you have benefited more from Singapore’s growth, and generally done better in life. But compared to workers younger than you, in their 30s and 40s today, you have generally earned less over your lifetimes. You have also had less time to benefit from improvements to the CPF syst

g) list documents

In [7]:
%%bash
curl -s 'http://localhost:12036/gen/v1/rag/collection/demo/documents'

[{"Id":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","CollectionName":"demo","ByteSize":43352,"FileName":"pm_long_speech_2023.txt","FileType":".txt","File":"PM Lee Hsien Loong delivered his National Day Rally speech on 20 August 2023 at the Institute of Technical Education (ITE) Headquarters. PM spoke in Malay and Chinese, followed by English.\n\n \n“A BETTER HOME, A BRIGHTER FUTURE”\nSECTION 1: INTRODUCTION\nMy fellow Singaporeans, good evening.\n\nReturn to Normalcy\nWe are all relieved that COVID is behind us. Life as we knew it has resumed.\n\nCOVID-19 was the most challenging ordeal for our nation since Independence. We can all be proud of how we pulled through together. Unlike many other countries, Singapore has emerged from the pandemic stronger, more resilient, and more united. This is a tribute to the indomitable spirit of our nation.\n\nGlobal situation and economic outlook\nHaving come through COVID, we are once again being tested. The international environment is fraught wi

h) get document

In [8]:
%%bash
curl -s 'http://localhost:12036/gen/v1/rag/collection/demo/document/PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U'

{"Id":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","CollectionName":"demo","ByteSize":43352,"FileName":"pm_long_speech_2023.txt","FileType":".txt","File":null,"Source":"https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2023","Abstract":null,"Authors":null,"Title":null,"Publisher":null,"PublishedDate":null,"Comments":null,"Keywords":null,"CreatedAt":"2024-09-17T11:38:14.739628","UpdatedAt":"2024-09-17T11:38:14.739638","IsActive":true,"ChunkGroups":[{"Id":"8f677904-195a-45ff-b001-4f0cfaeceb7f","DocumentId":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","SplitAlgo":"recursive_split","ChunkCount":66,"ChunkSize":1000,"Overlap":100,"IsActive":true,"ChunksDir":"/tmp/chunks/d89830d0a8894e4ba3d4833938ab0a7b"}]}

i) update document

In [9]:
%%bash
curl -X PUT \
    http://localhost:12036/gen/v1/rag/collection/demo/document/PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U \
    -H 'Content-Type: application/json' \
    -s \
    -d '{
            "Publisher": "ABC"
        }'


{"message":"Document updated successfully","document":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U"}

j) delete document

In [10]:
%%bash
curl -s -X DELETE http://localhost:12036/gen/v1/rag/collection/demo/document/PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U

{"message":"Document with id PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U deleted successfully"}

---
#### Example : Index PDF using Multi-Step Indexing

The purpose for multi-step is to support interactive status update so that the client can make use of the websocket manager to get the status of the indexing process in a step-by-step manner.

NOTE: The same is achievable using the single-step indexing as well as before.

```
curl -X POST 'http://localhost:12036/gen/v1/rag/index-file' \
    -H 'accept: application/json' \
    -H 'Content-Type: multipart/form-data' \
    -s \
    -F 'collection_name=demo' \
    -F 'file=@./attention-is-all-you-need.pdf' \
    -F 'req={"CollectionName":"demo","Source": "https://arxiv.org/abs/1706.03762","FilePath":"attention-is-all-you-need.pdf"}'
```


##### Step 1: Index document header

In [7]:
%%bash
curl -X POST 'http://localhost:12036/gen/v1/rag/step/header' \
    -H 'accept: application/json' \
    -H 'Content-Type: multipart/form-data' \
    -s \
    -F 'file=@./attention-is-all-you-need.pdf' \
    -F 'req={"CollectionName":"demo","Source": "https://arxiv.org/abs/1706.03762","FilePath":"attention-is-all-you-need.pdf"}'

{"Id":"-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0","CollectionName":"demo","ByteSize":2215244,"FileName":"attention-is-all-you-need.pdf","FileType":".pdf","File":null,"Source":"https://arxiv.org/abs/1706.03762","Abstract":null,"Authors":null,"Title":null,"Publisher":null,"PublishedDate":null,"Comments":null,"Keywords":null,"CreatedAt":"2024-09-17T12:49:55.874967","UpdatedAt":"2024-09-17T12:49:55.874978","IsActive":true,"ChunkGroups":[]}

##### Step 2: Split document into a group of chunks

In [8]:
import json
import os

# Execute the curl command and capture the response
response = !curl -X POST 'http://localhost:12036/gen/v1/rag/step/split' \
    -H 'Content-Type: application/json' \
    -s \
    -d '{\
            "collection_name": "demo",\
            "document_id": "-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0",\
            "chunk_size": 1000,\
            "chunk_overlap": 100\
        }'

# Convert response to string, then load it as JSON
response_json = json.loads(''.join(response))

# Extract the Id
document_id = response_json["Id"]
print(document_id)

# Set environment variable
os.environ['DOCUMENT_ID'] = document_id


e6d6ac36-c92b-4dd0-a243-d7a1b9838636


##### Step 3: Index each chunk in the database

In [9]:
%%bash
echo $DOCUMENT_ID
curl -X POST 'http://localhost:12036/gen/v1/rag/step/index' \
    -H 'Content-Type: application/json' \
    -s \
    -d '{
            "collection_name": "demo",
            "document_id": "-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0",
            "chunkgroup_id": "'$DOCUMENT_ID'"
        }'

e6d6ac36-c92b-4dd0-a243-d7a1b9838636
{"DocumentId":"-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0","ChunkgroupId":"e6d6ac36-c92b-4dd0-a243-d7a1b9838636","ChunkIds":["8b7de58c-f0a6-4c58-bce6-8366326f5c5f","0a8b4b37-a9b4-4c2f-a7bf-b1cecebd274c","3a0c9227-8066-4298-8b44-543961fcdaa4","57ad9afa-25c0-4a9a-a26c-edba15abeaa4","98fd5c89-b15d-464a-ae9c-8a03e3de7200","337853c2-4cb7-4b08-9c2d-796c3502a1ea","2d384878-d168-465a-9ff5-4cf6939cd3b8","fd7f268b-9074-42bd-9e09-080875e102e7","32e5937d-ece2-4868-a70f-6bb6fa6ef19f","d065a27c-49ab-4d32-a1b6-168e3e816f7f","62e8f5a1-6e8f-4d5f-aa83-ccb6e1d3eb04","f124c405-a332-4e65-b680-3efe88775129","f84e0d32-52af-4bcc-86be-625ae26ff073","a2a56c37-dd83-4214-afa2-c7d4efc4925a","6eac33f4-7004-4da8-892b-4be11ecbb5f8","aba0262f-9b4c-4700-8807-08dbe9b78fad","09fa767d-0fb9-4f4c-b28b-b05ce1fa3494","f6768c6a-fb56-4acd-bb8d-21086e2d83e9","20ff4b8e-14df-499c-b2e4-4fc6e3f6bfef","6c5a5356-77b8-45f0-8968-da74c8e65c9e","0f3d4450-97b9-4320-a0aa-8ca9ab0f1795","dac5eb83-76b5-4ec

In [10]:
%%bash
curl -s 'http://localhost:12036/gen/v1/rag/collection/demo/documents'

[{"Id":"-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0","CollectionName":"demo","ByteSize":2215244,"FileName":"attention-is-all-you-need.pdf","FileType":".pdf","File":null,"Source":"https://arxiv.org/abs/1706.03762","Abstract":null,"Authors":null,"Title":null,"Publisher":null,"PublishedDate":null,"Comments":null,"Keywords":null,"CreatedAt":"2024-09-17T12:49:55.874967","UpdatedAt":"2024-09-17T12:49:55.874978","IsActive":true,"ChunkGroups":[{"Id":"e6d6ac36-c92b-4dd0-a243-d7a1b9838636","DocumentId":"-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0","SplitAlgo":"recursive_split","ChunkCount":44,"ChunkSize":1000,"Overlap":100,"IsActive":true,"ChunksDir":"/tmp/chunks/266f5e8901cc4d9e9b7c12134e112969"}]}]

In [11]:
%%bash
curl -s 'http://localhost:12036/gen/v1/rag/collection/demo/document/-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0'

{"Id":"-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0","CollectionName":"demo","ByteSize":2215244,"FileName":"attention-is-all-you-need.pdf","FileType":".pdf","File":null,"Source":"https://arxiv.org/abs/1706.03762","Abstract":null,"Authors":null,"Title":null,"Publisher":null,"PublishedDate":null,"Comments":null,"Keywords":null,"CreatedAt":"2024-09-17T12:49:55.874967","UpdatedAt":"2024-09-17T12:49:55.874978","IsActive":true,"ChunkGroups":[{"Id":"e6d6ac36-c92b-4dd0-a243-d7a1b9838636","DocumentId":"-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0","SplitAlgo":"recursive_split","ChunkCount":44,"ChunkSize":1000,"Overlap":100,"IsActive":true,"ChunksDir":"/tmp/chunks/266f5e8901cc4d9e9b7c12134e112969"}]}

In [13]:
%%bash
curl -X GET "http://localhost:12036/gen/v1/rag/collection/demo/document/-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0/file" \
     -H "accept: application/octet-stream" \
     --output document_file.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2163k  100 2163k    0     0  35.5M      0 --:--:-- --:--:-- --:--:-- 35.8M


---

## 5. Docker

**Instructions:** 

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **docker: build**

- get updated version number from pyproject.toml

- update docker-compose.yml gai-rag-svr image tag with the new version number.

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **docker-compose: up**

#### Smoke Test

In [16]:
%%bash
curl http://localhost:12036


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    25  100    25    0     0   2012      0 --:--:-- --:--:-- --:--:--  2083


{"message":"gai-rag-svr"}

**Tests:**

Repeat the API test (#)

**Tear Down:**

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **docker-compose: down**

### Debugging

a) Container must be started with "python -m debugpy --listen 0.0.0.0:5678 main.py"

b) Port 5678 must be opened.

c) Click on "Debug" in Tool bar

d) Select "Attach" > "Run and Debug"

e) Add a "breakpoint" in the code

f) Run the API test to see if it trigger the breakpoint.